1. Извлечение аудиодорожки из видео

In [ ]:
!pip install moviepy
!pip install ffmpeg
from moviepy.editor import *

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=9b1ac75cdfa3e9b0ef9b0928d1a765200c53765bbe895192d78ed67d2ebfe060
  Stored in directory: /root/.cache/pip/wheels/56/30/c5/576bdd729f3bc062d62a551be7fefd6ed2f761901568171e4e
Successfully built ffmpeg


  if event.key is 'enter':



In [ ]:
video = VideoFileClip("video.mp4")

In [ ]:
video.audio.write_audiofile("audio.wav")

2. Преобразование речи в текст

In [ ]:
!pip install SpeechRecognition pydub

import speech_recognition as sr
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 40.9 MB/s eta 0:00:00


In [ ]:
sound = AudioSegment.from_wav("audio.wav")
sound.export("audio.wav", format="wav")

In [ ]:
# инициализация распознавателя
r = sr.Recognizer()

In [ ]:
import re

In [ ]:
# функция для распознавания речи в аудиофайле
def transcribe_audio(path):
    # используем аудиофайл как источник аудио
    with sr.AudioFile(path) as source:
        audio_listened = r.record(source)
        # преобразуем его в текст
        text = r.recognize_google(audio_listened, language="es-ES")
    return text

In [ ]:
# функция для деления аудиозаписи на отрывки по паузам
# + запуск распознавания
def get_large_audio_transcription_on_silence(path):
    """Делим большой аудиофайл на отрывки и применяем распознавание речи
    на каждом из отрывков (у Гугл в бесплатной версии есть ограничение по величине аудио)"""
    # открываем аудио с помощью pydub
    sound = AudioSegment.from_file(path)
    # делим звук на отрывки там, где пауза больше 500 милисекунд
    chunks = split_on_silence(sound,
        # настраиваемый параметр
        min_silence_len = 500,
        # настраиваемый параметр
        silence_thresh = sound.dBFS-12,
        # добавляем паузу по 1 сек. с начала и с конца, настраиваемый параметр
        keep_silence=1000,
    )
    folder_name = "audio-chunks"
    # создаем папку для хранения отрывков аудио
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # обрабатываем каждый отрывок
    for i, audio_chunk in enumerate(chunks, start=1):
        # экспортируем каждый отрывок и сохраняем его в созданной папке
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # распознаем отрывок
        try:
            text = transcribe_audio(chunk_filename)
        except sr.UnknownValueError as e:
            print(".", str(e))
        else:
            text = f"{text.capitalize()}.\n"
            print(text)
            whole_text += text
    # возвращаем текст для всех распознанных отрывков
    return whole_text

In [ ]:
path = "audio.wav"


In [ ]:
with open("text.txt", "w") as file:
  file.write(get_large_audio_transcription_on_silence(path))

3. Расставление знаков препинания.

In [ ]:
import os
import yaml
import torch
from torch import package
# инициализация модели
torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)

with open('latest_silero_models.yml', 'r') as yaml_file:
    models = yaml.load(yaml_file, Loader=yaml.SafeLoader)
model_conf = models.get('te_models').get('latest')

In [ ]:
model_url = model_conf.get('package')

model_dir = "downloaded_model"
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, os.path.basename(model_url))

if not os.path.isfile(model_path):
    torch.hub.download_url_to_file(model_url,
                                   model_path,
                                   progress=True)

imp = package.PackageImporter(model_path)
model = imp.load_pickle("te_model", "model")

def apply_te(text, lan='es'):
    return model.enhance_text(text, lan)

In [ ]:
with open('text.txt', 'r') as f:
  input_text = f.read()

with open("text_punkt.txt", "w") as output_text:
    output_text.write(apply_te(input_text, lan='es'))
